Idea:
1. Each number is independent. Hence, the total epetitions for each num will be likely to be equal in a long term.
2. Focus on the relations between a number to the rest. For example, if number 1 is drawn, what are the chances for the other particular number to be drawn based on the previous draws.
3. So each number will have a dictionary of other 44 numbers while each number has two parameters: Winning and Supps.
4. Later, give more weights to those have higher chances when generating a random ticket number

The process:
1. Scan the previous draws and list the dictionary for the number relations.
2. Setting some threshold and weights to generate a random ticket
3. Each play has 10 tickets
4. Check the results: Hypothesis tests

In [17]:
# Library Imports
import csv
import random
from enum import Enum

In [21]:
# Previous Lotto Records into Lists
wedLotto = satLotto = []
with open('Wednesday Lotto Results - G NETWORK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        wedLotto.append(row)

with open('Saturday Lotto Results - G NETWORK.csv', 'r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    for row in csv_reader:
        satLotto.append(row)
        
wedLotto = wedLotto[1::]
satLotto = satLotto[1::]

In [33]:
# Go through the previous draws and mark the relations between numbers
wedRelations = satRelations = {}

# list, set, dict are mutable -> pass by reference (if not reassign)
# ig. relationSetter(wedRelations, wedLotto)
def relationSetter(relations, prev_draws) -> dict:
    # String List to Int List
    for draw in prev_draws:
        try:
            drawnTicket = list(map(int, draw[1::]))
        except:
            continue
        for num in drawnTicket:
            for index in range (0, len(drawnTicket)):
                isSupp = True if index > 5 else False
                numNum = drawnTicket[index]
                if num != numNum:
                    if not num in relations:
                        relations[num] = {}
                    if not numNum in relations[num]:
                        relations[num][numNum] = {'winning':0, 'supp':0}
                    # Increment nested dictionary
                    if isSupp:
                        relations[num][numNum]['supp'] += 1
                    else:
                        relations[num][numNum]['winning'] += 1

In [34]:
relationSetter(wedRelations, wedLotto)



In [35]:
wedRelations

{8: {9: {'winning': 15, 'supp': 13},
  10: {'winning': 27, 'supp': 9},
  16: {'winning': 22, 'supp': 4},
  36: {'winning': 20, 'supp': 5},
  42: {'winning': 25, 'supp': 4},
  41: {'winning': 25, 'supp': 7},
  44: {'winning': 25, 'supp': 5},
  2: {'winning': 21, 'supp': 8},
  24: {'winning': 16, 'supp': 8},
  37: {'winning': 20, 'supp': 7},
  38: {'winning': 18, 'supp': 5},
  4: {'winning': 19, 'supp': 12},
  18: {'winning': 21, 'supp': 4},
  20: {'winning': 15, 'supp': 3},
  21: {'winning': 14, 'supp': 6},
  45: {'winning': 25, 'supp': 4},
  17: {'winning': 13, 'supp': 11},
  27: {'winning': 30, 'supp': 6},
  1: {'winning': 21, 'supp': 9},
  7: {'winning': 27, 'supp': 9},
  39: {'winning': 25, 'supp': 8},
  40: {'winning': 24, 'supp': 4},
  19: {'winning': 22, 'supp': 3},
  23: {'winning': 30, 'supp': 6},
  31: {'winning': 26, 'supp': 8},
  35: {'winning': 20, 'supp': 5},
  6: {'winning': 22, 'supp': 12},
  11: {'winning': 22, 'supp': 11},
  15: {'winning': 24, 'supp': 9},
  43: {'winn

In [ ]:
# enum for diff lotto
class Lotto(Enum):
    WED = 1
    SAT = 2
    
class Division(Enum):
    first = "First"
    second = "Second"
    third = "Third"
    fourth = "Fourth"
    fifth = "Fifth"
    sixth = "Sixth"
    failed = "Failed"
    

def ticketMarker(tickets, lotto, drawnNums) -> dict:
    winning_match = 0
    supp_match = 0
    # String List to Int List
    drawnTicket = list(map(int, wedLotto[1][1::]))
    results = {'First':0, 'Second':0, 'Third':0, 'Fourth':0,
              'Fifth':0, 'Sixth':0, 'Failed':0}
    for ticket in tickets:
        print(ticket)
        results[divisionCalc(ticket, lotto, drawnTicket)] += 1
    
    return results

    

def divisionCalc(ticket, lotto, drawnTicket) -> str:
    winning_match = 0
    supp_match = 0
    
    # To compare two lists -> need two indices
    drawnIndex = 0
    ticketIndex = 0
    while ticketIndex < len(ticket) and drawnIndex < len(ticket):
        ticketNum = ticket[ticketIndex]
        drawnNum = drawnTicket[drawnIndex]
        if ticketNum < drawnNum:
            # missed the drawnNum - drawnNum holds
            ticketIndex += 1
            continue 
        elif ticketNum == drawnNum:
            if index > 5: 
                supp_match += 1
            else:
                winnings_match += 1
            # Hit the drawnNum; Move to the next one
            drawnIndex += 1; ticketIndex += 1;
        else:
            # smaller than drawnNums - ticketNum holds
            drawnIndex += 1
    
    # Division Check
    # Rules - Common
    # 1. 6 winnings 2. 5 winnings + 1 supp 3. 5 winnings
    # 4. 4 winnings 
    if winning_match == 6:
        return Division.first.value
    elif winning_match == 5:
        if supp_match == 1:
            return Division.second.value
        else:
            return Division.third.value
    elif winning_match == 4:
        return  Division.fourth.value
    
    # Rules - Wed
    # 5. 3 winnings + 2 supp 6. total 3 (winning 1 + 2, 2 + 1)
    # Rules - Sat
    # 5. 3 winnings + 1 supp 6. 3 winnings
    if lotto == Lotto.WED:
        if winning_match == 3 and supp_match == 2:
            return Division.fifth.value
        else:
            if winning_match + supp_match == 3:
                return Division.sixth.value
            else:
                return Division.failed.value
    else:
        if winning_match == 3 and supp_match == 2:
            return Division.fifth.value
        else:
            if winning_match == 3:
                return Division.sixth.value
            else:
                return Division.failed.value
        

In [ ]:
tenTickets = []
for index in range(0,10):
    tenTickets.append(ticketGenerator())


ticketRow = wedLotto[0]
result = [ticketMarker(tenTickets, Lotto.SAT, ticketRow)]
result2 = ticketMarker(tenTickets, Lotto.SAT, ticketRow)
result.append(result2)


In [ ]:
#result = sorted(result, key = lambda k: k['First'])
result